# Protocol Benchmarks

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import *
from glob import glob as list_files
from pathlib import Path

In [ ]:
try:
    files = list_files(os.environ['BENCH_RESULTS_DIR'] + '/*/*.csv')
except:
    files = list_files("../bench-results/*/*.csv")
dfs = []
for file in files:
    global parent
    parent = Path(file).parent.name
    df = pd.read_csv(file, delimiter=";")
    df["runId"] = parent
    dfs.append(df)

df = pd.concat(dfs)
df["latency"] = df["latency"] / 1000
df["unit"] = "ms" # fix unit


for runId in df["runId"].unique():
    start_time = df[(df["runId"] == runId)]["send-time"].min()
    
    df.loc[df["runId"] == runId, "send-time"] = (df.loc[df["runId"] == runId, "send-time"] - start_time) / 1000
    df.loc[df["runId"] == runId, "receive-time"] = (df.loc[df["runId"] == runId, "receive-time"] - start_time) / 1000

df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,5))

for runId in df["runId"].unique():
    for name in df[(df["runId"] == runId)]["name"].unique():
        grouped = df[(df["runId"] == runId) & (df["name"] == name)].groupby({x: x // 10000 for x in range(len(df))})["latency"].mean().plot(ax=ax, label=f"{runId} {name}")
    
ax.set_xlabel("number of queries in groups of 10000")
ax.set_ylabel("latency [ms]")

ax.legend()

plt.savefig(fname="latency.png", format="png")

In [ ]:
df["seconds"] = (df["receive-time"] / 1000).apply(round)

fig, ax = plt.subplots(1, 1, figsize=(7,5))

for runId in df["runId"].unique():
    filtered_df = df[(df["runId"] == runId)]
    
    throughput = filtered_df.groupby("seconds")["name"].count()
    throughput.plot(ax=ax, label=runId)
    mean_throughput = throughput.quantile(0.5)
    # print(f"Mean Throughput for {runId}: {mean_throughput}")

    # also_mean_throughput = filtered_df["name"].count() / filtered_df["seconds"].max()
    # print(f"Diff {mean_throughput - also_mean_throughput}")
    
    ax.axhline(y=mean_throughput, color=ax.get_lines()[-1].get_color())
    # ax.axhline(y=also_mean_throughput, color=ax.get_lines()[-1].get_color())

ax.legend()

plt.savefig(fname="throughput.png", format="png")